In [1]:
import pandas as pd
import re

# credentials = {'client_id': 'client_a9523da7177b4bdda673b8b370299579',
#                'client_secret': 'secret_788f9e01feed51bfe832c7ffa62d8208'}

filename = 'local_listings.csv'
df = pd.read_csv(filename, sep=',')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [2]:
# df.head()


In [3]:
# Todo: Make sure all listings have a real price
# extract prices where available.
# identify listings with no price and then price range function
df['listing.priceDetails.displayPrice'] = df['listing.priceDetails.displayPrice'].fillna('none')
null_price = df['listing.priceDetails.price'].isnull()


In [4]:
# If the display price feature has a number, this is likely to be the price. 
# This is not expected to happen often.
display_is_number = df['listing.priceDetails.displayPrice'].str.isdigit()
df.loc[(null_price & display_is_number), 'listing.priceDetails.price'] = \
    df[null_price & display_is_number]['listing.priceDetails.displayPrice']




In [5]:
# replace the time and date parts of the display price before finding the price, so the numbers are only price

# Remove the numbers related to time
time_pattern = '[0-9]{1,2}\.[0-9]{1,2}[ap]'
df['time_in_price'] = df['listing.priceDetails.displayPrice'].str.findall(time_pattern)

# Replace the time in the display price
mask = ~df['time_in_price'].str[0].isnull()
df.loc[mask, 'listing.priceDetails.displayPrice'] = \
    df[mask]['listing.priceDetails.displayPrice'].str.replace(df['time_in_price'][df[mask].index[0]][0],'')


In [6]:
# Remove the dates from the price field
def remove_dates(df, pattern):
    df['date_in_price'] = df['listing.priceDetails.displayPrice'].str.findall(pattern, re.IGNORECASE)

    # Replace the date in the display price
    for idx, row in df.iterrows():
        if row['date_in_price']:
            removed_date = row['listing.priceDetails.displayPrice'].replace(row['date_in_price'][0],'')
            df.loc[idx, 'listing.priceDetails.displayPrice'] = removed_date

    return df

            
# Remove the numbers related to dates
date_pattern = r'(?=Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}'
df = remove_dates(df, date_pattern)

date_pattern = r'\d{1,2} (?=Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
df = remove_dates(df, date_pattern)

date_pattern = '[0-9]{1,2}[sSnNrRtT]'
df = remove_dates(df, date_pattern)

# Remove other phrases with numbers in them
# eg: 'sold in 7 days' or 'land 999m2'
pattern = '\d{1,2} (?=day)'
df = remove_dates(df, pattern)

pattern = '\d{2,4}\w2'
df = remove_dates(df, pattern)


In [7]:
df[['listing.priceDetails.displayPrice', 'displayPrice', 'fromPrice', 'toPrice']]

,listing.priceDetails.displayPrice,displayPrice,fromPrice,toPrice
0,Expressions of Interest close h June m,300,300.0,300.0
1,"Auction Guide $1,200,000",1200000,1200000.0,1200000.0
2,"Auction Guide $1,275,000",1275000,1275000.0,1275000.0
3,Auction - h June 9.00am,-900,900.0,900.0
4,"$1,300,000 - $1,400,000",1300000-1400000,1300000.0,1400000.0
5,"Guide $1,400,000",1400000,1400000.0,1400000.0
6,"Guide $1,400,000 - $1,500,000",1400000-1500000,1400000.0,1500000.0
7,price search,NaN,1425000.0,1400000.0
8,"Asking Price - $1,435,000",-1435000,1435000.0,1435000.0
9,"Asking Price $1,449,000",1449000,1449000.0,1449000.0


In [1017]:
def extend_numbers(df, pattern, delimiter=' ', number=0):

    df[f'alt{number}'] = df['listing.priceDetails.displayPrice'].str.findall(pattern, flags=re.IGNORECASE)    
    df['float_value'] = df[f'alt{number}'].str[0].str.split(delimiter).str[0].astype(float)*1e6
    df[f'replace_value_{number}'] = df['float_value'].fillna(0).astype(int)
    
    for idx, row in df.iterrows():

        if row[f'alt{number}']:
            extend_number = row['listing.priceDetails.displayPrice'].replace(row[f'alt{number}'][0],str(row[f'replace_value_{number}']))
            df.loc[idx, 'listing.priceDetails.displayPrice'] = extend_number

    
    return df

pattern = r'(\d{1}.{1,3} mill)'
df = extend_numbers(df, pattern)

pattern = r'\d{1,2} mill'
df = extend_numbers(df, pattern, number=1)

pattern = r'\d.\d{1,2}M$'
df = extend_numbers(df, pattern, delimiter='M', number=2)

# pattern = r'\d.\d{1,2} M$'
# df = extend_numbers(df, pattern, number=3)


In [1018]:
df['displayPrice'] = df['listing.priceDetails.displayPrice'].\
    str.findall(r'([0-9-]{1,3})').str.join(sep='')


In [1019]:
# df[null_price][['listing.priceDetails.displayPrice', 'listing.priceDetails.price', 
# #                 'listing.priceDetails.priceFrom','listing.priceDetails.priceTo', 
#                 'displayPrice']]



In [1020]:
df['fromPrice'] = df['listing.priceDetails.priceFrom']
df['toPrice'] = df['listing.priceDetails.priceTo']

df.loc[null_price, 'fromPrice'] = df[null_price]['displayPrice'].str.split('-').str[0]
df.loc[null_price, 'toPrice'] = df[null_price]['displayPrice'].str.split('-').str[1]

null_toPrice = df['toPrice'].isnull()
df.loc[null_toPrice, 'toPrice'] = df[null_toPrice]['fromPrice']

empty_price = df['toPrice'] == ''
df.loc[empty_price, 'toPrice'] = df[empty_price]['fromPrice']

empty_price = df['fromPrice'] == ''
df.loc[empty_price, 'fromPrice'] = df[empty_price]['toPrice']

# df[null_price][['listing.priceDetails.displayPrice', 'displayPrice', 'fromPrice', 'toPrice']]

,listing.priceDetails.displayPrice,displayPrice,fromPrice,toPrice
0,"UNDER OFFER $775,000",775000,775000,775000
1,"For Sale $1,050,000",1050000,1050000,1050000
2,"For Sale: $1,900,000 - $2,050,000",1900000-2050000,1900000,2050000
3,"Guide $1,200,000 - $1,320,000",1200000-1320000,1200000,1320000
4,"Price guide $3,750,000 | Inspect By Appointment",3750000,3750000,3750000
6,"Asking Price $2,950,000",2950000,2950000,2950000
7,Auction Saturday May,,,
8,Contact Agent,,,
9,"For Sale: $1,500,000 - $1,650,000",1500000-1650000,1500000,1650000
10,Contact Agent,,,


In [1021]:

df.to_csv('local_listings2.csv')
